# Amazon S3 Buckets
---
ECMWF open data can be retrieved from the Amazon S3 buckets using the [`earthkit`](https://earthkit-data.readthedocs.io/en/latest/guide/sources.html#s3) or `ecmwf-opendata` Python libraries.

Below, two examples for downloading data from the **Amazon's AWS** location.

In [ ]:
# !pip3 install earthkit ecmwf-opendata

In [ ]:
from ecmwf.opendata import Client

client = Client(source="aws")
request = {
    "time": 0,
    "type": "fc",
    "step": 24,
    "param": "2t",
}
client.retrieve(request, "aws_2t_data.grib2")
da_2t = ekd.from_source("file", "aws_2t_data.grib2")
da_2t.ls()

In [ ]:
import earthkit.data as ekd

data = ekd.from_source("s3", {
    "endpoint": "s3.amazonaws.com",
    "region": "eu-central-1",
    "bucket": "ecmwf-forecasts",
    "objects": "20230118/00z/0p4-beta/oper/20230118000000-0h-oper-fc.grib2"
}, anon=True)
ds = data.to_xarray()
ds

::::{important}
When you need to download historical data, bear in mind the file-naming convention.
:::{dropdown} File-naming convention
An example (for HH=`00z` and stream=`oper`) shows how a filename has changed since year 2023
* 20230118/00z/0p4-beta/oper/20230118000000-0h-oper-fc.grib2, <br> in Februar 2024 one can choose between the resolution `0p4-beta` and `0p25`
* 20240201/00z/$\color{red}{\text{0p4-beta}}$/oper/20240201000000-0h-oper-fc.grib2
* 20240201/00z/$\color{red}{\text{0p25}}$/oper/20240201000000-0h-oper-fc.grib2, <br> in March 2024 one can choose between `aifs` and `ifs` (only `ifs` is available in `0p4-beta` or `0p25`)
* 20240301/00z/$\color{red}{\text{aifs}}$/0p25/oper/20240301000000-0h-oper-fc.grib2
* 20240301/00z/$\color{red}{\text{ifs}}$/0p4-beta/oper/20240301000000-0h-oper-fc.grib2, <br> in February 2025 one can choose between `aifs-single` and `aifs` (`ifs` does not contain any changes)
* 20250210/00z/$\color{red}{\text{aifs-single}}$/0p25/$\color{red}{\text{experimental}}$/oper/20250210000000-0h-oper-fc.grib2
* 20250210/00z/$\color{red}{\text{aifs}}$/0p25/oper/20250210000000-0h-oper-fc.grib2, <br> in March 2025 the file-naming convention is the same as we know it today
* 20250301/00z/aifs-single/0p25/oper/20250301000000-0h-oper-fc.grib2
* 20250301/00z/ifs/0p25/oper/20250301000000-0h-oper-fc.grib2
:::
::::